# 🧠 BitterBot AI — ARC Prize 2025 Final Run
This notebook trains and evaluates the BitterBot ARC model **entirely within the Kaggle environment**.
- ✅ No Internet
- ✅ No warm start
- ✅ GPU-only (no CPU fallback)
- ✅ Auto-writes submission.csv for final scoring

In [ ]:
# --- Environment Setup ---
import os, random, json, torch, numpy as np
from pathlib import Path

def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

seed_all(42)
assert torch.cuda.is_available(), '❌ CUDA device required for final run'
device = torch.device('cuda')
print(f'✅ Using GPU: {torch.cuda.get_device_name(0)} (Total GPUs: {torch.cuda.device_count()})')

In [ ]:
# --- Dataset Paths ---
print('Available datasets in /kaggle/input:', os.listdir('/kaggle/input'))
ARC_TRAIN_PATH = '/kaggle/input/arc-agi-2/ARC-AGI-2-main/data/training/'
ARC_TEST_PATH  = '/kaggle/input/arc-agi-2/ARC-AGI-2-main/data/evaluation/'

assert os.path.exists(ARC_TRAIN_PATH), 'Training data path not found!'
assert os.path.exists(ARC_TEST_PATH), 'Eval data path not found!'
print('Training samples:', len(os.listdir(ARC_TRAIN_PATH)))
print('Evaluation samples:', len(os.listdir(ARC_TEST_PATH)))

In [ ]:
# --- Import BitterBot ---
import sys
sys.path.append('/kaggle/input/bitterbot-v01-v1')

# Add progress bar and ETA tracking
from tqdm.auto import tqdm

print('🚀 Starting training...')
epochs = 150
for epoch in tqdm(range(epochs), desc='Training Epochs'):
    train_model(model,
                data_path=ARC_TRAIN_PATH,
                epochs=1,
                save_dir='/kaggle/working/checkpoints',
                epoch_offset=epoch)
print('✅ All epochs complete.')
from models.topas_arc_60M import TopasARC60M
from trainers.train_parent import train_model
from validation.eval_ttt import evaluate_tasks

print('✅ BitterBot modules imported successfully.')

In [ ]:
# --- Train from Scratch (No Warm Start) ---
model = TopasARC60M().to(device)
model.train()

# Add progress bar and ETA tracking
from tqdm.auto import tqdm

print('🚀 Starting training...')
epochs = 150
for epoch in tqdm(range(epochs), desc='Training Epochs'):
    train_model(model,
                data_path=ARC_TRAIN_PATH,
                epochs=1,
                save_dir='/kaggle/working/checkpoints',
                epoch_offset=epoch)
print('✅ All epochs complete.')
            data_path=ARC_TRAIN_PATH,
            epochs=150,
            save_dir='/kaggle/working/checkpoints')

print('🏁 Training complete and checkpoint saved.')

In [ ]:
# --- Evaluation and Submission ---
model.eval()
checkpoint = '/kaggle/working/checkpoints/final_epoch150.pt'
torch.save(model.state_dict(), checkpoint)
print('Checkpoint verified:', os.path.exists(checkpoint))

evaluate_tasks(model,
               data_path=ARC_TEST_PATH,
               output_file='/kaggle/working/submission.csv')

print('✅ Evaluation finished. Submission ready at /kaggle/working/submission.csv')

In [ ]:
# --- Validate Submission ---
import pandas as pd
sub_path = '/kaggle/working/submission.csv'
assert os.path.exists(sub_path), 'Submission file not created!'
df = pd.read_csv(sub_path)
print('Submission preview:')
display(df.head())
print(f'Total rows: {len(df)}, File size: {os.path.getsize(sub_path)/1024:.2f} KB')
print('🎯 Notebook complete — Ready to commit for final scoring.')